In [5]:
import pandas as pd
import random
import numpy as np
from utilities.api_functions import *

# load the lookup table
geoid_lu_df = pd.read_csv('/Users/joshpaul/epa-justice/repo/epa-justice/tbl/geoid_lookup.csv')
# get a random GVV ID for testing
gvv_id = geoid_lu_df['id'].iloc[random.sample(range(len(geoid_lu_df['id'])), 1)].values[0]

In [6]:
dhc = fetch_data_and_compute("dhc", gvv_id, geoid_lu_df)
acs5 = fetch_data_and_compute("acs5", gvv_id, geoid_lu_df)

fetching data from: https://api.census.gov/data/2020/dec/dhc?get=P12_001N,P12_002N,P12_026N,P12_003N,P12_004N,P12_005N,P12_006N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_027N,P12_028N,P12_029N,P12_030N,P12_044N,P12_045N,P12_046N,P12_047N,P12_048N,P12_049N&for=place:56600&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
fetching data from: https://api.census.gov/data/2020/acs/acs5/subject?get=S1810_C03_001E,S1810_C03_001M,S2701_C03_001E,S2701_C03_001M,S2701_C05_001E,S2701_C05_001M&for=place:56600&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706


In [7]:
dhc

,total_population,pct_65_plus,pct_under_18
0,217.0,5.529954,43.778802


In [8]:
acs5

,pct_w_disability,moe_pct_w_disability,pct_insured,moe_pct_insured,pct_uninsured,moe_pct_uninsured
0,16.4,8.0,73.0,13.3,27.0,13.3
